# Most Polluted Countries - Limpieza de Datos

### · Antes de empezar... 

1. Para gestionar los Dataframes:

In [1]:
#Para mi doc de Pipeline:
import suport as sp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
from fuzzywuzzy import process, fuzz
import pickle

/Users/paulaelizagarate/opt/anaconda3/envs/Bootcamp/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
#Regex
import re

###  · Carga de CSVs

Empezamos cargando datos CSV procedentes de Kaggle con la información de los países y las ciudades más contaminadas desde 2017 hasta 2021. 

In [3]:
df_pcountries = pd.read_csv("datos/AIR QUALITY INDEX- top countries.csv", index_col = 0 )
df_pcountries.head(2)

,Country/Region,2021,2020,2019,2018,Population
Rank,,,,,,
1,Bangladesh,76.9,77.1,83.3,97.1,"164,689,383"
2,Chad,75.9,-,-,-,"16,425,859"


In [228]:
df_pcountries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 1 to 118
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country/Region  118 non-null    object 
 1   2021            118 non-null    float64
 2   2020            118 non-null    object 
 3   2019            118 non-null    object 
 4   2018            118 non-null    object 
 5   Population      118 non-null    object 
dtypes: float64(1), object(5)
memory usage: 6.5+ KB


In [4]:
#df_pcountries.values

In [5]:
df_pcities = pd.read_csv("datos/AIR QUALITY INDEX (by cities) - IQAir.csv", index_col = 0 )
df_pcities.head(2)

,City,2021,JAN(2021),FEB(2021),MAR(2021),APR(2021),MAY(2021),JUN(2021),JUL(2021),AUG(2021),SEP(2021),OCT(2021),NOV(2021),DEC(2021),2020,2019,2018,2017
Rank,,,,,,,,,,,,,,,,,,
1,"Bhiwadi, India",106.2,145.8,129.8,120.2,125.7,86.5,95.9,55.6,55.4,37.1,91.1,188.6,136.6,95.5,83.4,125.4,-
2,"Ghaziabad, India",102.0,199.9,172.2,97.8,86.3,52.9,47.2,35.3,37.6,30.8,89.7,218.3,163,106.6,110.2,135.2,144.6


A continuación, cargamos nuestros CSVs con los datos scrapeados acerca de la natalidad en el mismo periodo. 

In [6]:
df_n17= pd.read_csv("datos/natalidad/natalidad2017.csv" ,  index_col = 0)
df_n17.head(5)

,0
1,"España [+] 2017 393.181 202.478 190.703 8,41‰ ..."
2,"Alemania [+] 2017 784.901 402.517 382.384 9,50..."
3,Reino Unido [+] 2017 754.754 387.030 367.724 1...
4,"Francia [+] 2017 770.045 394.058 375.987 11,50..."
5,"Italia [+] 2017 458.151 235.733 222.418 7,60‰ ..."


In [7]:
df_n18= pd.read_csv("datos/natalidad/natalidad2018.csv" ,  index_col = 0)
df_n18.head(5)

,0
1,"España [+] 2018 372.777 191.569 181.208 7,94‰ ..."
2,"Alemania [+] 2018 787.523 404.052 383.471 9,50..."
3,Reino Unido [+] 2018 730.918 375.379 355.539 1...
4,"Francia [+] 2018 759.199 388.944 370.255 11,30..."
5,"Italia [+] 2018 439.747 226.217 213.530 7,30‰ ..."


In [8]:
df_n19= pd.read_csv("datos/natalidad/natalidad2019.csv" ,  index_col = 0)
df_n19.head(5)

,0
1,"España [+] 2019 360.617 185.523 175.094 7,62‰ ..."
2,"Alemania [+] 2019 778.090 399.292 378.798 9,40..."
3,Reino Unido [+] 2019 712.699 375.379 355.539 1...
4,"Francia [+] 2019 754.008 385.340 368.668 11,20..."
5,"Italia [+] 2019 420.084 215.387 204.697 7,00‰ ..."


In [9]:
df_n20= pd.read_csv("datos/natalidad/natalidad2020.csv" ,  index_col = 0)
df_n20.head(5)

,0
1,"España [+] 2020 341.315 174.842 166.473 7,19‰ ..."
2,"Alemania [+] 2020 773.144 397.385 375.759 9,30..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2020 735.775 376.187 359.588 10,90..."
5,"Italia [+] 2020 404.892 208.431 196.461 6,80‰ ..."


In [10]:
df_n21= pd.read_csv("datos/natalidad/natalidad2021.csv" ,  index_col = 0)
df_n21.head(5)

,0
1,"España [+] 2021 337.380 174.148 163.232 7,12‰ ..."
2,"Alemania [+] 2021 795.492 397.385 375.759 9,60..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2021 738.594 376.187 359.588 10,90..."
5,"Italia [+] 2021 399.431 208.431 196.461 6,80‰ ..."


### · Limpieza Datos Scrapeados 

In [11]:
df_n17

,0
1,"España [+] 2017 393.181 202.478 190.703 8,41‰ ..."
2,"Alemania [+] 2017 784.901 402.517 382.384 9,50..."
3,Reino Unido [+] 2017 754.754 387.030 367.724 1...
4,"Francia [+] 2017 770.045 394.058 375.987 11,50..."
5,"Italia [+] 2017 458.151 235.733 222.418 7,60‰ ..."
...,...
189,"Samoa [+] 2017 24,69‰ 3,93 -1,26%"
190,"Yemen [+] 2017 31,04‰ 3,89 -2,60%"
191,"Sudáfrica [+] 2017 20,91‰ 2,43 -1,10%"
192,"Zambia [+] 2017 36,70‰ 4,72 -1,99%"


Empezaremos por limpiar el grueso de nuestros datos, discriminando los países. La decisión de hacerlo de esta manera es porque el split de la mayoría de información lo haremos por espacios y en los países no es válido ya que muchos tienen nombre compuesto.

In [12]:
df_n17= df_n17["0"].str.split("\s\[", regex= True, expand=True)
#df_n17

Para hacer la división del grueso de datos, crearemos un diccionario. Aprovecharemos que la estructura de los datos extraídos es fija para realizar el almacenamiento en el mismo. Lo primero es conocer la longitud de los diferentes paquetes de información, para crear los condicionales de los que dependerá este almacenamiento. En aquellos datos que no estén informados, con el objetivo de mantener la estructura, introduciremos un valor nulo. Este diccionario será nuestro puente para los DataFrames finales. 

In [13]:
lon=[]

for i in df_n17[1]:
    lon.append(len(i.split()))

In [14]:
lon[:5]

[8, 8, 8, 8, 8]

In [15]:
numeros_unicos = set(lon)

print(numeros_unicos)

{8, 5, 6, 7}


In [16]:
#lon.index(7)

Una vez sabemos la longitud de nuestros distintos paquetes de datos, empezamos con la creación del diccionario:

In [17]:
nat2017 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n17[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(str(a[3]))
        nat2017["female_born"].append(str(a[4]))
        nat2017["birth_rate"].append(str(a[5]))
        nat2017["fertility_rate"].append(str(a[6]))
        nat2017["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(str(a[3]))
        nat2017["female_born"].append(str(a[4]))
        nat2017["birth_rate"].append(str(a[5]))
        nat2017["fertility_rate"].append(str(a[6]))
        nat2017["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(str(a[2]))
        nat2017["male_born"].append(np.nan)
        nat2017["female_born"].append(np.nan)
        nat2017["birth_rate"].append(str(a[3]))
        nat2017["fertility_rate"].append(str(a[4]))
        nat2017["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2017["year"].append(str(a[1]))
        nat2017["born"].append(np.nan)
        nat2017["male_born"].append(np.nan)
        nat2017["female_born"].append(np.nan)
        nat2017["birth_rate"].append(str(a[2]))
        nat2017["fertility_rate"].append(str(a[3]))
        nat2017["variation"].append(str(a[4]))
            

In [18]:
#control
len(nat2017["year"])

193

Una vez finalizado nuestro diccionario con el grueso de datos, debemos hacer la limpieza de la columna países. La limpieza se basará en eliminar espacios o "y" en los nombres, en su lugar pondremos "_": 

In [19]:
df_n17[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [20]:
df_n17[0] = df_n17[0].apply(lambda x: str(x).replace(" y ","_"))
df_n17[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [21]:
df_n17[0] = df_n17[0].apply(lambda x: str(x).replace(" ","_"))
df_n17[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

Ya disponemos de los países limpios por lo que están listos para ser unidos al resto del diccionario. Esto nos formará uno de nuestros cinco DataFrames finales. Haremos un diccionario con los países limpios, que después zipearemos con la información de nuestro anterior diccionario:

In [22]:
paises17={
        "country": []
            }

for i in df_n17[0]:
    
    paises17["country"].append(i)

In [23]:
#paises17

In [24]:
union17 = list(zip(paises17["country"], nat2017["year"], nat2017["born"], nat2017["male_born"], nat2017["female_born"], nat2017["birth_rate"], nat2017["fertility_rate"] , nat2017["variation"]))

###### *PUNTO DE CONTROL* 📍
Necesario para reseteo DataFrame

In [25]:
n17= pd.DataFrame(union17)
#n17

In [26]:
#control
#n17.info()
#n17.isnull().sum()

Nuestro siguiente paso será dar nombre a las columnas:

In [27]:
n17.columns

RangeIndex(start=0, stop=8, step=1)

In [28]:
n17.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/526978396.py:1: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  n17.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


El siguiente paso será gestionar los tipos de datos que tenemos. Nos interesa tener la mayor cantidad de datos numéricos posibles:

In [29]:
n17["year"]= n17["year"].astype("int")

In [30]:
n17["born"] = n17["born"].str.replace(".", "")

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/2687544233.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  n17["born"] = n17["born"].str.replace(".", "")


In [31]:
n17["born"] = pd.to_numeric(n17["born"])

In [32]:
n17["male_born"] = n17["male_born"].str.replace(".","")

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/2191722072.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  n17["male_born"] = n17["male_born"].str.replace(".","")


In [33]:
n17["male_born"] = pd.to_numeric(n17["male_born"])

In [34]:
n17["female_born"] = n17["female_born"].str.replace(".","")

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3584901649.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  n17["female_born"] = n17["female_born"].str.replace(".","")


In [35]:
n17["female_born"] = pd.to_numeric(n17["female_born"])

##### Gestión de porcentaje  

La gestión a realizar del porcentaje es su eliminación para evitar que sean objetos los números. Primero lo que haremos es sustituir las comas, por puntos, así nos permitirá tener un `float`. Acto seguido eliminados finalmente el porcentaje.  

In [36]:
n17["birth_rate"]= n17["birth_rate"].str.replace("‰","")

In [37]:
n17["birth_rate"]= n17["birth_rate"].str.replace(",","")

In [38]:
n17["birth_rate"] = pd.to_numeric(n17["birth_rate"])

In [39]:
n17["fertility_rate"] = n17["fertility_rate"].str.replace("%","")

In [40]:
n17["fertility_rate"] = n17["fertility_rate"].str.replace(",",".")

In [41]:
n17["fertility_rate"]= pd.to_numeric(n17["fertility_rate"])

In [42]:
n17["variation"] = n17["variation"].str.replace("%","")

In [43]:
n17["variation"]= n17["variation"].str.replace("‰","")

In [44]:
n17["variation"] = n17["variation"].str.replace(",",".")

In [45]:
n17["variation"]= pd.to_numeric(n17["variation"])

In [46]:
#n17

In [47]:
#control para saber como va mi transformación de datos
#n17.info()

#### ¡Lo tenemos! ✨

Primer DataFrame limpio y listo para ser guardado para nuestra fase de analísis. 

In [48]:
n17.to_csv('n17_def.csv')

Ahora a por los otros 4 años de información, donde repetiremos el proceso realizado. Para agilizar el proceso, crearemos funciones de limpieza que iremos aplicando a nuestra información:

### 2018

In [49]:
df_n18

,0
1,"España [+] 2018 372.777 191.569 181.208 7,94‰ ..."
2,"Alemania [+] 2018 787.523 404.052 383.471 9,50..."
3,Reino Unido [+] 2018 730.918 375.379 355.539 1...
4,"Francia [+] 2018 759.199 388.944 370.255 11,30..."
5,"Italia [+] 2018 439.747 226.217 213.530 7,30‰ ..."
...,...
189,"Samoa [+] 2018 24,38‰ 3,88 -1,25%"
190,"Yemen [+] 2018 30,45‰ 3,79 -2,52%"
191,"Sudáfrica [+] 2018 20,51‰ 2,41 -1,03%"
192,"Zambia [+] 2018 36,19‰ 4,63 -1,80%"


In [50]:
df_n18= df_n18["0"].str.split("\s\[", regex= True, expand=True)
#df_n18

In [51]:
lon18=[]

for i in df_n18[1]:
    lon18.append(len(i.split()))

In [52]:
unicos18 = set(lon18)

print(unicos18)

{8, 5, 6, 7}


In [53]:
nat2018 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n18[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(str(a[3]))
        nat2018["female_born"].append(str(a[4]))
        nat2018["birth_rate"].append(str(a[5]))
        nat2018["fertility_rate"].append(str(a[6]))
        nat2018["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(str(a[3]))
        nat2018["female_born"].append(str(a[4]))
        nat2018["birth_rate"].append(str(a[5]))
        nat2018["fertility_rate"].append(str(a[6]))
        nat2018["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(str(a[2]))
        nat2018["male_born"].append(np.nan)
        nat2018["female_born"].append(np.nan)
        nat2018["birth_rate"].append(str(a[3]))
        nat2018["fertility_rate"].append(str(a[4]))
        nat2018["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2018["year"].append(str(a[1]))
        nat2018["born"].append(np.nan)
        nat2018["male_born"].append(np.nan)
        nat2018["female_born"].append(np.nan)
        nat2018["birth_rate"].append(str(a[2]))
        nat2018["fertility_rate"].append(str(a[3]))
        nat2018["variation"].append(str(a[4]))

In [54]:
#control
len(nat2018["year"])

193

In [55]:
df_n18[0] = df_n18[0].apply(lambda x: str(x).replace(" y ","_"))
df_n18[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [56]:
df_n18[0] = df_n18[0].apply(lambda x: str(x).replace(" ","_"))
df_n18[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [57]:
paises18={
        "country": []
            }

for i in df_n18[0]:
    
    paises18["country"].append(i)

In [58]:
union18 = list(zip(paises18["country"], nat2018["year"], nat2018["born"], nat2018["male_born"], nat2018["female_born"], nat2018["birth_rate"], nat2018["fertility_rate"] , nat2018["variation"]))

#### *PUNTO DE CONTROL* 📍
Necesario para reseteo DataFrame

In [59]:
n18= pd.DataFrame(union18)
#n18

###### Limpieza datos
Nuestra primera función definida será para establecer las cabeceras de nuestras columnas:

In [60]:
def titulos_col(x):
    x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)

In [61]:
titulos_col(n18)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/2230106971.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


Nuestras siguientes funciones a definir serán las que nos ayudaran a convertir nuestros objetos en números:

In [62]:
n18["year"]= n18["year"].astype("int")

In [63]:
def numeros(x):
    x = x.str.replace(".","")
    x = pd.to_numeric(x)
    return(x)

In [64]:
sp.numeros(n18["born"])

/Users/paulaelizagarate/Bootcamp/ETL_project/suport.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


0      372777.0
1      787523.0
2      730918.0
3      759199.0
4      439747.0
         ...   
188         NaN
189         NaN
190         NaN
191         NaN
192         NaN
Name: born, Length: 193, dtype: float64

In [65]:
n18["born"] = numeros(n18["born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [66]:
n18["male_born"]= numeros(n18["male_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [67]:
n18["female_born"]= numeros(n18["female_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


Por último haremos una función que limpie nuestros porcentajes, buscando de nuevo que queden en formato número para poder manejarlos:

In [68]:
def porcentajes (x):
    x = x.str.replace("‰","")
    x = x.str.replace("%","")
    x = x.str.replace(",",".")
    x = pd.to_numeric(x)
    return(x)

In [69]:
n18["birth_rate"]= porcentajes(n18["birth_rate"])

In [70]:
n18["fertility_rate"]= porcentajes(n18["fertility_rate"])

In [71]:
n18["variation"]= porcentajes(n18["variation"])

In [72]:
#n18

In [73]:
#control para saber como va la transformación de datos
#n18.info()

Ahora solo tendríamos que seguir aplicando nuestras funciones a nuestros DataFrames para terminar esta limpieza 🧽🪣  

### 2019

In [74]:
df_n19

,0
1,"España [+] 2019 360.617 185.523 175.094 7,62‰ ..."
2,"Alemania [+] 2019 778.090 399.292 378.798 9,40..."
3,Reino Unido [+] 2019 712.699 375.379 355.539 1...
4,"Francia [+] 2019 754.008 385.340 368.668 11,20..."
5,"Italia [+] 2019 420.084 215.387 204.697 7,00‰ ..."
...,...
189,"Samoa [+] 2019 24,10‰ 3,83 -1,21%"
190,"Yemen [+] 2019 29,87‰ 3,70 -2,43%"
191,"Sudáfrica [+] 2019 20,13‰ 2,38 -1,00%"
192,"Zambia [+] 2019 35,78‰ 4,56 -1,60%"


In [75]:
df_n19= df_n19["0"].str.split("\s\[", regex= True, expand=True)
#df_n19

In [76]:
lon19=[]

for i in df_n19[1]:
    lon19.append(len(i.split()))

In [77]:
unicos19 = set(lon19)

print(unicos19)

{8, 5, 6, 7}


In [78]:
nat2019 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n19[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(str(a[3]))
        nat2019["female_born"].append(str(a[4]))
        nat2019["birth_rate"].append(str(a[5]))
        nat2019["fertility_rate"].append(str(a[6]))
        nat2019["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(str(a[3]))
        nat2019["female_born"].append(str(a[4]))
        nat2019["birth_rate"].append(str(a[5]))
        nat2019["fertility_rate"].append(str(a[6]))
        nat2019["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(str(a[2]))
        nat2019["male_born"].append(np.nan)
        nat2019["female_born"].append(np.nan)
        nat2019["birth_rate"].append(str(a[3]))
        nat2019["fertility_rate"].append(str(a[4]))
        nat2019["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2019["year"].append(str(a[1]))
        nat2019["born"].append(np.nan)
        nat2019["male_born"].append(np.nan)
        nat2019["female_born"].append(np.nan)
        nat2019["birth_rate"].append(str(a[2]))
        nat2019["fertility_rate"].append(str(a[3]))
        nat2019["variation"].append(str(a[4]))

In [79]:
#control
len(nat2019["year"])

193

In [80]:
df_n19[0] = df_n19[0].apply(lambda x: str(x).replace(" y ","_"))
df_n19[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [81]:
df_n19[0] = df_n19[0].apply(lambda x: str(x).replace(" ","_"))
df_n19[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [82]:
paises19={
        "country": []
            }

for i in df_n19[0]:
    
    paises19["country"].append(i)

In [83]:
union19 = list(zip(paises19["country"], nat2019["year"], nat2019["born"], nat2019["male_born"], nat2019["female_born"], nat2019["birth_rate"], nat2019["fertility_rate"] , nat2019["variation"]))

In [84]:
n19= pd.DataFrame(union19)
#n19

##### Limpiamos Datos 

In [85]:
titulos_col(n19)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/2230106971.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


In [86]:
n19["year"]= n19["year"].astype("int")

In [87]:
n19["born"]= numeros(n19["born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [88]:
n19["male_born"]= numeros(n19["male_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [89]:
n19["female_born"]= numeros(n19["female_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [90]:
n19["birth_rate"] = porcentajes(n19["birth_rate"])

In [91]:
n19["fertility_rate"] = porcentajes(n19["fertility_rate"])

In [92]:
n19["variation"] = porcentajes(n19["variation"])

In [93]:
n19

,country,year,born,male_born,female_born,birth_rate,fertility_rate,variation
0,España,2019,360617.0,185523.0,175094.0,7.62,1.24,-2.24
1,Alemania,2019,778090.0,399292.0,378798.0,9.40,1.54,-1.91
2,Reino_Unido,2019,712699.0,375379.0,355539.0,10.70,1.63,-2.98
3,Francia,2019,754008.0,385340.0,368668.0,11.20,1.86,-0.53
4,Italia,2019,420084.0,215387.0,204697.0,7.00,1.27,-1.55
...,...,...,...,...,...,...,...,...
188,Samoa,2019,NaN,NaN,NaN,24.10,3.83,-1.21
189,Yemen,2019,NaN,NaN,NaN,29.87,3.70,-2.43
190,Sudáfrica,2019,NaN,NaN,NaN,20.13,2.38,-1.00
191,Zambia,2019,NaN,NaN,NaN,35.78,4.56,-1.60


In [94]:
#control de como va mi transformación de datos:
#n19.info()

### 2020

In [95]:
df_n20

,0
1,"España [+] 2020 341.315 174.842 166.473 7,19‰ ..."
2,"Alemania [+] 2020 773.144 397.385 375.759 9,30..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2020 735.775 376.187 359.588 10,90..."
5,"Italia [+] 2020 404.892 208.431 196.461 6,80‰ ..."
...,...
189,"Samoa [+] 2020 23,86‰ 3,79 -1,17%"
190,"Yemen [+] 2020 29,30‰ 3,61 -2,32%"
191,"Sudáfrica [+] 2020 19,77‰ 2,36 -0,97%"
192,"Zambia [+] 2020 35,44‰ 4,50 -1,38%"


In [96]:
df_n20= df_n20["0"].str.split("\s\[", regex= True, expand=True)
df_n20

,0,1
1,España,"+] 2020 341.315 174.842 166.473 7,19‰ 1,19 -4,03%"
2,Alemania,"+] 2020 773.144 397.385 375.759 9,30‰ 1,53 -0,65%"
3,Reino Unido,"+] 2020 712.699 375.379 355.539 10,20‰ 1,56 -4..."
4,Francia,"+] 2020 735.775 376.187 359.588 10,90‰ 1,83 -1..."
5,Italia,"+] 2020 404.892 208.431 196.461 6,80‰ 1,24 -2,36%"
...,...,...
189,Samoa,"+] 2020 23,86‰ 3,79 -1,17%"
190,Yemen,"+] 2020 29,30‰ 3,61 -2,32%"
191,Sudáfrica,"+] 2020 19,77‰ 2,36 -0,97%"
192,Zambia,"+] 2020 35,44‰ 4,50 -1,38%"


In [97]:
lon20=[]

for i in df_n20[1]:
    lon20.append(len(i.split()))

In [98]:
unicos20 = set(lon20)

print(unicos20)

{8, 5, 6, 7}


In [99]:
nat2020 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n20[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(str(a[3]))
        nat2020["female_born"].append(str(a[4]))
        nat2020["birth_rate"].append(str(a[5]))
        nat2020["fertility_rate"].append(str(a[6]))
        nat2020["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(str(a[3]))
        nat2020["female_born"].append(str(a[4]))
        nat2020["birth_rate"].append(str(a[5]))
        nat2020["fertility_rate"].append(str(a[6]))
        nat2020["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(str(a[2]))
        nat2020["male_born"].append(np.nan)
        nat2020["female_born"].append(np.nan)
        nat2020["birth_rate"].append(str(a[3]))
        nat2020["fertility_rate"].append(str(a[4]))
        nat2020["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2020["year"].append(str(a[1]))
        nat2020["born"].append(np.nan)
        nat2020["male_born"].append(np.nan)
        nat2020["female_born"].append(np.nan)
        nat2020["birth_rate"].append(str(a[2]))
        nat2020["fertility_rate"].append(str(a[3]))
        nat2020["variation"].append(str(a[4]))

In [100]:
#control
len(nat2020["year"])

193

In [101]:
df_n20[0] = df_n20[0].apply(lambda x: str(x).replace(" y ","_"))
df_n20[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [102]:
df_n20[0] = df_n20[0].apply(lambda x: str(x).replace(" ","_"))
df_n20[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [103]:
paises20={
        "country": []
            }

for i in df_n20[0]:
    
    paises20["country"].append(i)

In [104]:
union20 = list(zip(paises20["country"], nat2020["year"], nat2020["born"], nat2020["male_born"], nat2020["female_born"], nat2020["birth_rate"], nat2020["fertility_rate"] , nat2020["variation"]))

In [105]:
n20= pd.DataFrame(union20)
n20

,0,1,2,3,4,5,6,7
0,España,2020,341.315,174.842,166.473,"7,19‰","1,19","-4,03%"
1,Alemania,2020,773.144,397.385,375.759,"9,30‰","1,53","-0,65%"
2,Reino_Unido,2020,712.699,375.379,355.539,"10,20‰","1,56","-4,29%"
3,Francia,2020,735.775,376.187,359.588,"10,90‰","1,83","-1,61%"
4,Italia,2020,404.892,208.431,196.461,"6,80‰","1,24","-2,36%"
...,...,...,...,...,...,...,...,...
188,Samoa,2020,NaN,NaN,NaN,"23,86‰","3,79","-1,17%"
189,Yemen,2020,NaN,NaN,NaN,"29,30‰","3,61","-2,32%"
190,Sudáfrica,2020,NaN,NaN,NaN,"19,77‰","2,36","-0,97%"
191,Zambia,2020,NaN,NaN,NaN,"35,44‰","4,50","-1,38%"


##### *Limpiamos Datos* 

In [106]:
titulos_col(n20)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/2230106971.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


In [107]:
n20["year"]= n20["year"].astype("int")

In [108]:
n20["born"] = numeros(n20["born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [109]:
n20["male_born"]= numeros(n20["male_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [110]:
n20["female_born"]= numeros(n20["female_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [111]:
n20["birth_rate"]= porcentajes(n20["birth_rate"])

In [112]:
n20["fertility_rate"]= porcentajes(n20["fertility_rate"])

In [113]:
n20["variation"]= porcentajes(n20["variation"])

In [139]:
#n20

In [115]:
#control transformación datos
#n20.info()

### 2021

In [116]:
df_n21

,0
1,"España [+] 2021 337.380 174.148 163.232 7,12‰ ..."
2,"Alemania [+] 2021 795.492 397.385 375.759 9,60..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2021 738.594 376.187 359.588 10,90..."
5,"Italia [+] 2021 399.431 208.431 196.461 6,80‰ ..."
...,...
189,"Samoa [+] 2020 23,86‰ 3,79 -1,17%"
190,"Yemen [+] 2020 29,30‰ 3,61 -2,32%"
191,"Sudáfrica [+] 2020 19,77‰ 2,36 -0,97%"
192,"Zambia [+] 2020 35,44‰ 4,50 -1,38%"


In [117]:
df_n21= df_n21["0"].str.split("\s\[", regex= True, expand=True)
#df_n21

In [118]:
lon21=[]

for i in df_n21[1]:
    lon21.append(len(i.split()))

In [119]:
unicos21 = set(lon21)

print(unicos21)

{8, 5, 6, 7}


In [120]:
nat2021 = {
    "year": [],
    "born": [],
    "male_born": [],
    "female_born":[],
    "birth_rate": [],
    "fertility_rate":[],
    "variation":[]
            }

data_list = df_n21[1]

for data in data_list:
    
    a = data.split()
    
    if len(a)== 8:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(str(a[3]))
        nat2021["female_born"].append(str(a[4]))
        nat2021["birth_rate"].append(str(a[5]))
        nat2021["fertility_rate"].append(str(a[6]))
        nat2021["variation"].append(str(a[7]))
    
    if len(a)== 7:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(str(a[3]))
        nat2021["female_born"].append(str(a[4]))
        nat2021["birth_rate"].append(str(a[5]))
        nat2021["fertility_rate"].append(str(a[6]))
        nat2021["variation"].append(np.nan)    
            
    if len(a)== 6:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(str(a[2]))
        nat2021["male_born"].append(np.nan)
        nat2021["female_born"].append(np.nan)
        nat2021["birth_rate"].append(str(a[3]))
        nat2021["fertility_rate"].append(str(a[4]))
        nat2021["variation"].append(str(a[5]))
            
    if len(a)== 5:
        nat2021["year"].append(str(a[1]))
        nat2021["born"].append(np.nan)
        nat2021["male_born"].append(np.nan)
        nat2021["female_born"].append(np.nan)
        nat2021["birth_rate"].append(str(a[2]))
        nat2021["fertility_rate"].append(str(a[3]))
        nat2021["variation"].append(str(a[4]))

In [121]:
#control
len(nat2021["year"])

193

In [122]:
df_n21[0] = df_n21[0].apply(lambda x: str(x).replace(" y ","_"))
df_n21[0].unique()[:5]

array(['España', 'Alemania', 'Reino Unido', 'Francia', 'Italia'],
      dtype=object)

In [123]:
df_n21[0] = df_n21[0].apply(lambda x: str(x).replace(" ","_"))
df_n21[0].unique()[:5]

array(['España', 'Alemania', 'Reino_Unido', 'Francia', 'Italia'],
      dtype=object)

In [124]:
paises21={
        "country": []
            }

for i in df_n21[0]:
    
    paises21["country"].append(i)

In [125]:
union21 = list(zip(paises21["country"], nat2021["year"], nat2021["born"], nat2021["male_born"], nat2021["female_born"], nat2021["birth_rate"], nat2021["fertility_rate"] , nat2021["variation"]))

In [126]:
n21= pd.DataFrame(union21)
#n21

##### Limpiamos Datos 

In [127]:
titulos_col(n21)

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/2230106971.py:2: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  x.set_axis(["country", "year", "born", "male_born", "female_born", "birth_rate", "fertility_rate", "variation"], axis=1, inplace=True)


In [128]:
n21["year"]= n21["year"].astype("int")

In [129]:
n21["born"] = numeros(n21["born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [130]:
n21["male_born"]= numeros(n21["male_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [131]:
n21["female_born"]= numeros(n21["female_born"])

/var/folders/8l/zm2f15815wb7zxjgwnj0zkjh0000gn/T/ipykernel_1166/3039227266.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  x = x.str.replace(".","")


In [132]:
n21["birth_rate"]= porcentajes(n21["birth_rate"])

In [133]:
n21["fertility_rate"]= porcentajes(n21["fertility_rate"])

In [134]:
n21["variation"]= porcentajes(n21["variation"])

In [143]:
#n21

In [136]:
#control para saber como llevo la transformación de datos
#n21.info()

### Concatenación Tablas Natalidad 

Como punto final, concatenaremos las cinco tablas que hemos limpiado para formar una única tabla con los datos de natalidad comprendidos entre 2017/2021:

In [140]:
natalidad_total= pd.concat([n17, n18, n19, n20, n21], axis=0, ignore_index = True)

In [224]:
natalidad_total

,country,year,born,male_born,female_born,birth_rate,fertility_rate,variation
0,spain,2017,393181.0,202478.0,190703.0,841.00,1.31,-1.92
1,germany,2017,784901.0,402517.0,382384.0,950.00,1.57,-1.88
2,united_kingdom,2017,754754.0,387030.0,367724.0,1140.00,1.74,-2.79
3,france,2017,770045.0,394058.0,375987.0,1150.00,1.89,-1.56
4,italy,2017,458151.0,235733.0,222418.0,760.00,1.32,-1.49
...,...,...,...,...,...,...,...,...
960,samoa,2020,NaN,NaN,NaN,23.86,3.79,-1.17
961,yemen,2020,NaN,NaN,NaN,29.30,3.61,-2.32
962,south_africa,2020,NaN,NaN,NaN,19.77,2.36,-0.97
963,zambia,2020,NaN,NaN,NaN,35.44,4.50,-1.38


In [144]:
#control todos los registros en tabla final
lentotal= len(n21) + len(n20) + len(n19) + len(n18) + len(n17)
lentotal

965

### Traducción Países DataFrame

In [195]:
pip install deep_translator

Note: you may need to restart the kernel to use updated packages.


In [196]:
from deep_translator import GoogleTranslator

In [197]:
traductor= GoogleTranslator(source='es', target='en')

In [206]:
country= list(natalidad_total["country"])

In [210]:
resultado = traductor.translate_batch(country)

In [214]:
natalidad_total["country"]= resultado

Volvemos a trabajar el formato de la columna países ya que al traducir lo hemos perdido.  

In [216]:
natalidad_total["country"]= natalidad_total["country"].apply(lambda x: str(x).replace(" ","_"))

In [219]:
natalidad_total["country"].unique()

array(['Spain', 'Germany', 'United_Kingdom', 'France', 'Italy',
       'Portugal', 'USA', 'Japan', 'China', 'Andorra',
       'United_Arab_Emirates', 'Afghanistan', 'Antigua_Barbuda',
       'Albanian', 'Armenia', 'Angola', 'Argentina', 'Austria',
       'Australia', 'Azerbaijan', 'Bosnia_Herzegovina', 'barbados',
       'Bangladesh', 'Belgium', 'Burkina_Faso', 'Bulgaria', 'bahrain',
       'Burundi', 'Benin', 'Brunei', 'bolivian', 'Brazil', 'Bahamas',
       'bhutan', 'botswana', 'belarus', 'belize', 'Canada',
       'Democratic_Republic_of_Congo', 'Central_African_Republic',
       'Republic_of_Congo', 'Swiss', 'Ivory_Coast', 'Chili', 'Cameroon',
       'Colombia', 'Costa_Rica', 'Cuba', 'Cape_Verde', 'Cyprus',
       'czech_republic', 'djibouti', 'Denmark', 'Dominica',
       'Dominican_Republic', 'Algeria', 'Ecuador', 'Estonia', 'Egypt',
       'eritrea', 'Ethiopia', 'Finland', 'fiji',
       'Federated_States_of_Micronesia', 'Gabon', 'Grenade', 'Georgia',
       'Ghana', 'Gambia', 

In [220]:
def minusculas (x):
    x= [i.lower() for i in x]
    return(x)

In [223]:
natalidad_total["country"]= minusculas(natalidad_total["country"])

Ahora si que si, tenemos nuestros datos de natalidad listos para ser utilizados. Los guardaremos para utilizarlos más adelante.  

In [225]:
natalidad_total.to_csv('natalidad_total.csv')